In [1]:
import cmlreaders as cml
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')
pd.options.display.max_columns=None
pd.options.display.max_rows=100
from IPython.display import display, Markdown, HTML
import pandas_to_pybeh as pb
from tqdm import tqdm_notebook as lol
import time

## Hospital Data Load

In [5]:
evs = []
data = cml.get_data_index(kind = 'r1'); data = data[data['experiment'] == 'RepFR2']
for subject, df in data.groupby('subject'):
    for session in pd.unique(df['session']):
        r = cml.CMLReader(subject=subject, experiment='RepFR2', session=session)
        temp = r.load('task_events')
        evs.append(temp)
evs = pd.concat(evs)
evs = evs[evs.list != 0]

,eegoffset,eegfile,experiment,intruded,intrusion,is_repeat,is_stim,item_name,item_num,list,montage,msoffset,mstime,phase,protocol,recalled,rectime,repeats,serialpos,session,stim_list,stim_params,subject,type
50,430736,R1579T_RepFR2_0_25Aug21_1727,RepFR2,0,-999,False,False,,-999,1,0,-1,1629930871035,retrieval,r1,False,-999,-999,-999,0,True,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",R1579T,TRIAL_START
51,445563,R1579T_RepFR2_0_25Aug21_1727,RepFR2,0,-999,False,False,,-999,1,0,-1,1629930885862,encoding,r1,False,-999,-999,-999,0,True,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",R1579T,COUNTDOWN
52,449754,R1579T_RepFR2_0_25Aug21_1727,RepFR2,0,-999,False,True,GLOBE,105,1,0,-1,1629930890053,encoding,r1,True,-999,3,0,0,True,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",R1579T,WORD
53,449756,R1579T_RepFR2_0_25Aug21_1727,RepFR2,0,-999,False,False,,-999,1,0,-1,1629930890055,encoding,r1,False,-999,-999,-999,0,True,"{'_remove': 0, 'amplitude': 0.5, 'anode_label'...",R1579T,STIM
54,452113,R1579T_RepFR2_0_25Aug21_1727,RepFR2,0,-999,False,True,FROG,97,1,0,-1,1629930892412,encoding,r1,True,-999,3,1,0,True,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",R1579T,WORD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,2237431,R1590T_RepFR2_1_16Nov21_1240,RepFR2,0,-999,False,False,,-999,20,0,-1,1637090290761,retrieval,r1,False,-999,-999,-999,1,True,"{'_remove': 0, 'amplitude': 0.5, 'anode_label'...",R1590T,STIM
1172,2241180,R1590T_RepFR2_1_16Nov21_1240,RepFR2,0,-999,False,False,,-999,20,0,-1,1637090294510,retrieval,r1,False,-999,-999,-999,1,True,"{'_remove': 0, 'amplitude': 0.5, 'anode_label'...",R1590T,STIM
1173,2245170,R1590T_RepFR2_1_16Nov21_1240,RepFR2,0,-999,False,False,,-999,20,0,-1,1637090298500,retrieval,r1,False,-999,-999,-999,1,True,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",R1590T,REC_END
1174,2245217,R1590T_RepFR2_1_16Nov21_1240,RepFR2,0,-999,False,False,,-999,21,0,-1,1637090298547,retrieval,r1,False,-999,-999,-999,1,True,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",R1590T,SESS_END


In [4]:
rec_stim = evs[evs.phase == 'retrieval']
rec_stim = rec_stim[rec_stim.type == 'STIM']
rec_stim['inter_stim_time'] = rec_stim.mstime.diff()

rec_stim
for i, df in rec_stim.groupby(['subject', 'session', 'list']):
    sub, sess, l = i
    ist = np.delete(df.inter_stim_time.to_numpy(), 0)
    print(i, ist, len(ist))


('R1579T', 0, 4) [4902. 2364. 2383. 7287. 2487. 2395.] 6
('R1579T', 0, 7) [2512. 2539. 2430. 4920. 2505. 2408. 2451. 2567.] 8
('R1579T', 0, 11) [2450. 2420. 4847. 2591. 2575. 2468. 2535.] 7
('R1579T', 0, 12) [4902. 7484. 2583. 7502. 2539.] 5
('R1579T', 0, 13) [4815. 7427. 2360. 7449.] 4
('R1579T', 0, 14) [2358. 2395. 2339. 2545. 4943. 2363. 2550. 2446. 2473.] 9
('R1579T', 0, 15) [2485. 2447. 4906. 2563. 5133.] 5
('R1579T', 0, 16) [4718.] 1
('R1579T', 0, 18) [5019. 2450. 2561. 2355. 4809. 2376. 2516.] 7
('R1579T', 0, 20) [5066. 5185. 2593. 2491. 4916.] 5
('R1585E', 0, 3) [3550. 3751. 3751. 3750. 3749. 3751. 3751.] 7
('R1585E', 0, 4) [3554. 3750. 3753. 3750. 3748. 3751. 3752.] 7
('R1585E', 0, 6) [3550. 3755. 3746. 3749. 3756. 3751. 3746.] 7
('R1585E', 0, 7) [3551. 3749. 3751. 3751. 3751. 3735. 3749.] 7
('R1585E', 0, 8) [3552. 3750. 3751. 3749. 3751. 3751. 3750.] 7
('R1585E', 0, 11) [ 3551.  3751.  3750.  3752.  3733. 49207.] 6
('R1587J', 0, 1) [3555. 3751. 3751. 3751. 3749. 3750. 3751.] 